In [ ]:
#!/usr/bin/env python3
"""
E-commerce Sales Analysis & Forecasting
Comprehensive data analysis project for data analyst portfolio
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os

warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

class EcommerceSalesAnalysis:
    def __init__(self):
        self.df = None
        self.output_dir = 'outputs'
        os.makedirs(self.output_dir, exist_ok=True)

    def generate_data(self, n_records=10000):
        """
        Generate realistic e-commerce sales data
        """
        print("\n=== Generating E-commerce Sales Data ===")
        np.random.seed(42)

        # Date range: 2 years of data
        start_date = datetime(2023, 1, 1)
        dates = [start_date + timedelta(days=x) for x in range(730)]

        # Generate data
        data = {
            'order_id': [f'ORD{str(i).zfill(6)}' for i in range(1, n_records + 1)],
            'order_date': np.random.choice(dates, n_records),
            'category': np.random.choice(
                ['Technology', 'Furniture', 'Office Supplies', 'Clothing', 'Electronics'],
                n_records,
                p=[0.25, 0.20, 0.25, 0.15, 0.15]
            ),
            'region': np.random.choice(
                ['East', 'West', 'North', 'South'],
                n_records,
                p=[0.30, 0.35, 0.20, 0.15]
            ),
            'customer_segment': np.random.choice(
                ['Consumer', 'Corporate', 'Home Office'],
                n_records,
                p=[0.50, 0.30, 0.20]
            ),
            'quantity': np.random.randint(1, 10, n_records),
        }

        self.df = pd.DataFrame(data)

        # Generate price based on category
        price_ranges = {
            'Technology': (200, 2000),
            'Furniture': (100, 1500),
            'Office Supplies': (10, 200),
            'Clothing': (20, 300),
            'Electronics': (50, 1000)
        }

        self.df['unit_price'] = self.df['category'].apply(
            lambda x: np.random.uniform(*price_ranges[x])
        )

        # Calculate sales
        self.df['sales'] = self.df['quantity'] * self.df['unit_price']

        # Add discount (0-30%)
        self.df['discount'] = np.random.choice([0, 5, 10, 15, 20, 25, 30], n_records, p=[0.3, 0.2, 0.2, 0.15, 0.1, 0.03, 0.02])
        self.df['sales'] = self.df['sales'] * (1 - self.df['discount']/100)

        # Calculate profit (15-40% margin depending on category and discount)
        base_margin = {'Technology': 0.30, 'Furniture': 0.25, 'Office Supplies': 0.35, 'Clothing': 0.40, 'Electronics': 0.28}
        self.df['profit_margin'] = self.df['category'].map(base_margin) - (self.df['discount'] / 200)
        self.df['profit'] = self.df['sales'] * self.df['profit_margin']

        # Add some seasonality (higher sales in Q4)
        self.df['month'] = pd.to_datetime(self.df['order_date']).dt.month
        seasonal_multiplier = self.df['month'].apply(lambda x: 1.5 if x in [11, 12] else 1.2 if x in [6, 7] else 1.0)
        self.df['sales'] = self.df['sales'] * seasonal_multiplier
        self.df['profit'] = self.df['profit'] * seasonal_multiplier

        # Sort by date
        self.df = self.df.sort_values('order_date').reset_index(drop=True)

        print(f"✓ Generated {len(self.df):,} records")
        print(f"✓ Date range: {self.df['order_date'].min().date()} to {self.df['order_date'].max().date()}")

        return self.df

    def data_overview(self):
        """
        Display basic data overview and statistics
        """
        print("\n=== DATA OVERVIEW ===")
        print(f"\nDataset Shape: {self.df.shape[0]} rows × {self.df.shape[1]} columns")
        print("\nColumn Data Types:")
        print(self.df.dtypes)
        print("\nMissing Values:")
        print(self.df.isnull().sum())
        print("\n" + "="*50)

    def descriptive_statistics(self):
        """
        Calculate and display descriptive statistics
        """
        print("\n=== DESCRIPTIVE STATISTICS ===")

        numeric_cols = ['sales', 'quantity', 'profit', 'discount']
        stats_summary = self.df[numeric_cols].describe()
        print("\n", stats_summary)

        # Key business metrics
        print("\n=== KEY BUSINESS METRICS ===")
        total_revenue = self.df['sales'].sum()
        total_profit = self.df['profit'].sum()
        avg_order_value = self.df['sales'].mean()
        profit_margin_pct = (total_profit / total_revenue) * 100

        print(f"Total Revenue: ${total_revenue:,.2f}")
        print(f"Total Profit: ${total_profit:,.2f}")
        print(f"Average Order Value: ${avg_order_value:,.2f}")
        print(f"Overall Profit Margin: {profit_margin_pct:.2f}%")
        print(f"Total Orders: {len(self.df):,}")

        # Save to file
        with open(f'{self.output_dir}/sales_analysis_report.txt', 'w') as f:
            f.write("E-COMMERCE SALES ANALYSIS REPORT\n")
            f.write("="*60 + "\n\n")
            f.write("KEY BUSINESS METRICS\n")
            f.write("-"*60 + "\n")
            f.write(f"Total Revenue: ${total_revenue:,.2f}\n")
            f.write(f"Total Profit: ${total_profit:,.2f}\n")
            f.write(f"Average Order Value: ${avg_order_value:,.2f}\n")
            f.write(f"Overall Profit Margin: {profit_margin_pct:.2f}%\n")
            f.write(f"Total Orders: {len(self.df):,}\n")

        return total_revenue, total_profit

    def exploratory_data_analysis(self):
        """
        Comprehensive EDA with visualizations
        """
        print("\n=== EXPLORATORY DATA ANALYSIS ===")

        # 1. Sales by Category
        print("\n1. Analyzing sales by category...")
        category_sales = self.df.groupby('category').agg({
            'sales': 'sum',
            'profit': 'sum',
            'order_id': 'count'
        }).round(2)
        category_sales.columns = ['Total Sales', 'Total Profit', 'Order Count']
        category_sales = category_sales.sort_values('Total Sales', ascending=False)
        print(category_sales)

        # Visualization
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # Plot 1: Sales by Category
        category_sales['Total Sales'].plot(kind='bar', ax=axes[0, 0], color='steelblue')
        axes[0, 0].set_title('Total Sales by Category', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Category')
        axes[0, 0].set_ylabel('Sales ($)')
        axes[0, 0].tick_params(axis='x', rotation=45)

        # 2. Sales by Region
        print("\n2. Analyzing sales by region...")
        region_sales = self.df.groupby('region')['sales'].sum().sort_values(ascending=False)
        print(region_sales)

        # Plot 2: Sales by Region
        region_sales.plot(kind='bar', ax=axes[0, 1], color='coral')
        axes[0, 1].set_title('Total Sales by Region', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Region')
        axes[0, 1].set_ylabel('Sales ($)')
        axes[0, 1].tick_params(axis='x', rotation=45)

        # 3. Sales by Customer Segment
        print("\n3. Analyzing sales by customer segment...")
        segment_sales = self.df.groupby('customer_segment')['sales'].sum().sort_values(ascending=False)
        print(segment_sales)

        # Plot 3: Customer Segment Distribution
        axes[0, 1].clear()
        segment_sales.plot(kind='pie', ax=axes[0, 1], autopct='%1.1f%%', startangle=90)
        axes[0, 1].set_title('Sales Distribution by Customer Segment', fontsize=14, fontweight='bold')
        axes[0, 1].set_ylabel('')

        # 4. Discount Analysis
        print("\n4. Analyzing discount impact...")
        discount_analysis = self.df.groupby('discount').agg({
            'sales': 'mean',
            'profit': 'mean',
            'order_id': 'count'
        }).round(2)
        print(discount_analysis)

        # Plot 4: Discount vs Profit
        axes[1, 0].scatter(self.df['discount'], self.df['profit'], alpha=0.5)
        axes[1, 0].set_title('Discount vs Profit Analysis', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Discount (%)')
        axes[1, 0].set_ylabel('Profit ($)')

        # 5. Profit Margin Distribution
        axes[1, 1].hist(self.df['profit_margin']*100, bins=30, color='green', alpha=0.7, edgecolor='black')
        axes[1, 1].set_title('Profit Margin Distribution', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Profit Margin (%)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].axvline(self.df['profit_margin'].mean()*100, color='red', linestyle='--', label=f'Mean: {self.df["profit_margin"].mean()*100:.1f}%')
        axes[1, 1].legend()

        plt.tight_layout()
        plt.savefig(f'{self.output_dir}/01_eda_overview.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {self.output_dir}/01_eda_overview.png")
        plt.close()

    def time_series_analysis(self):
        """
        Analyze sales trends over time
        """
        print("\n=== TIME SERIES ANALYSIS ===")

        # Convert to datetime
        self.df['order_date'] = pd.to_datetime(self.df['order_date'])

        # Monthly aggregation
        monthly_data = self.df.groupby(self.df['order_date'].dt.to_period('M')).agg({
            'sales': 'sum',
            'profit': 'sum',
            'order_id': 'count'
        })
        monthly_data.index = monthly_data.index.to_timestamp()

        print("\nMonthly Sales Summary:")
        print(monthly_data.tail())

        # Visualizations
        fig, axes = plt.subplots(3, 1, figsize=(16, 12))

        # Plot 1: Monthly Sales Trend
        axes[0].plot(monthly_data.index, monthly_data['sales'], marker='o', linewidth=2, markersize=6)
        axes[0].set_title('Monthly Sales Trend', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Month')
        axes[0].set_ylabel('Sales ($)')
        axes[0].grid(True, alpha=0.3)

        # Add trend line
        z = np.polyfit(range(len(monthly_data)), monthly_data['sales'], 1)
        p = np.poly1d(z)
        axes[0].plot(monthly_data.index, p(range(len(monthly_data))), "r--", alpha=0.8, linewidth=2, label='Trend')
        axes[0].legend()

        # Plot 2: Monthly Profit Trend
        axes[1].plot(monthly_data.index, monthly_data['profit'], marker='s', color='green', linewidth=2, markersize=6)
        axes[1].set_title('Monthly Profit Trend', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Month')
        axes[1].set_ylabel('Profit ($)')
        axes[1].grid(True, alpha=0.3)

        # Plot 3: Order Volume Trend
        axes[2].bar(monthly_data.index, monthly_data['order_id'], color='orange', alpha=0.7)
        axes[2].set_title('Monthly Order Volume', fontsize=14, fontweight='bold')
        axes[2].set_xlabel('Month')
        axes[2].set_ylabel('Number of Orders')
        axes[2].grid(True, alpha=0.3, axis='y')

        plt.tight_layout()
        plt.savefig(f'{self.output_dir}/02_time_series_analysis.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {self.output_dir}/02_time_series_analysis.png")
        plt.close()

        return monthly_data

    def correlation_analysis(self):
        """
        Analyze correlations between numerical variables
        """
        print("\n=== CORRELATION ANALYSIS ===")

        # Select numerical columns
        num_cols = ['sales', 'quantity', 'discount', 'profit', 'profit_margin']
        correlation_matrix = self.df[num_cols].corr()

        print("\nCorrelation Matrix:")
        print(correlation_matrix.round(3))

        # Visualization
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                    square=True, linewidths=1, fmt='.2f', cbar_kws={"shrink": 0.8})
        plt.title('Correlation Matrix - Sales Data', fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.savefig(f'{self.output_dir}/03_correlation_matrix.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {self.output_dir}/03_correlation_matrix.png")
        plt.close()

    def hypothesis_testing(self):
        """
        Perform statistical hypothesis testing
        """
        print("\n=== HYPOTHESIS TESTING ===")

        # Test 1: Does discount significantly affect sales?
        print("\n1. Testing: Does discount affect sales?")
        high_discount = self.df[self.df['discount'] >= 15]['sales']
        low_discount = self.df[self.df['discount'] < 15]['sales']

        t_stat, p_value = stats.ttest_ind(high_discount, low_discount)
        print(f"   T-statistic: {t_stat:.4f}")
        print(f"   P-value: {p_value:.4f}")
        if p_value < 0.05:
            print("   ✓ Result: Significant difference found (p < 0.05)")
        else:
            print("   ✗ Result: No significant difference (p >= 0.05)")

        # Test 2: Sales difference across regions
        print("\n2. Testing: Sales difference across regions (ANOVA)")
        regions = [self.df[self.df['region'] == r]['sales'] for r in self.df['region'].unique()]
        f_stat, p_value = stats.f_oneway(*regions)
        print(f"   F-statistic: {f_stat:.4f}")
        print(f"   P-value: {p_value:.4f}")
        if p_value < 0.05:
            print("   ✓ Result: Significant difference across regions (p < 0.05)")
        else:
            print("   ✗ Result: No significant difference (p >= 0.05)")

    def sales_forecasting(self, monthly_data):
        """
        Build simple sales forecasting model
        """
        print("\n=== SALES FORECASTING ===")

        # Prepare data
        monthly_data = monthly_data.reset_index()
        monthly_data['month_num'] = range(len(monthly_data))

        # Train-test split (80-20)
        split_idx = int(len(monthly_data) * 0.8)
        train = monthly_data[:split_idx]
        test = monthly_data[split_idx:]

        # Build model
        X_train = train[['month_num']].values
        y_train = train['sales'].values
        X_test = test[['month_num']].values
        y_test = test['sales'].values

        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predictions
        y_pred = model.predict(X_test)

        # Evaluate
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"\nModel Performance:")
        print(f"  RMSE: ${rmse:,.2f}")
        print(f"  MAE: ${mae:,.2f}")
        print(f"  R² Score: {r2:.4f}")

        # Forecast next 6 months
        future_months = np.array([[i] for i in range(len(monthly_data), len(monthly_data) + 6)])
        future_sales = model.predict(future_months)

        print(f"\nForecast for next 6 months:")
        for i, sales in enumerate(future_sales, 1):
            print(f"  Month {i}: ${sales:,.2f}")

        # Visualization
        plt.figure(figsize=(14, 6))
        plt.plot(train['month_num'], train['sales'], 'o-', label='Training Data', linewidth=2)
        plt.plot(test['month_num'], test['sales'], 's-', label='Test Data (Actual)', linewidth=2)
        plt.plot(test['month_num'], y_pred, '^--', label='Test Data (Predicted)', linewidth=2)
        plt.plot(future_months, future_sales, 'd--', label='Forecast', linewidth=2, markersize=8)
        plt.xlabel('Month Number', fontsize=12)
        plt.ylabel('Sales ($)', fontsize=12)
        plt.title('Sales Forecasting - Linear Regression Model', fontsize=14, fontweight='bold')
        plt.legend(fontsize=11)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{self.output_dir}/04_sales_forecast.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {self.output_dir}/04_sales_forecast.png")
        plt.close()

        # Save forecast to CSV
        forecast_df = pd.DataFrame({
            'Month': range(1, 7),
            'Forecasted_Sales': future_sales
        })
        forecast_df.to_csv(f'{self.output_dir}/forecast_results.csv', index=False)
        print(f"✓ Saved: {self.output_dir}/forecast_results.csv")

    def advanced_visualizations(self):
        """
        Create additional insightful visualizations
        """
        print("\n=== CREATING ADVANCED VISUALIZATIONS ===")

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # 1. Category vs Region Heatmap
        pivot_data = self.df.pivot_table(values='sales', index='category', columns='region', aggfunc='sum')
        sns.heatmap(pivot_data, annot=True, fmt='.0f', cmap='YlOrRd', ax=axes[0, 0], cbar_kws={'label': 'Sales ($)'})
        axes[0, 0].set_title('Sales Heatmap: Category × Region', fontsize=14, fontweight='bold')

        # 2. Box plot: Sales by Category
        self.df.boxplot(column='sales', by='category', ax=axes[0, 1])
        axes[0, 1].set_title('Sales Distribution by Category', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Category')
        axes[0, 1].set_ylabel('Sales ($)')
        plt.sca(axes[0, 1])
        plt.xticks(rotation=45)

        # 3. Quantity vs Sales scatter
        axes[1, 0].scatter(self.df['quantity'], self.df['sales'], alpha=0.5, c=self.df['discount'], cmap='viridis')
        axes[1, 0].set_title('Quantity vs Sales (Color: Discount)', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Quantity')
        axes[1, 0].set_ylabel('Sales ($)')
        cbar = plt.colorbar(axes[1, 0].collections[0], ax=axes[1, 0])
        cbar.set_label('Discount (%)')

        # 4. Top 10 Days by Sales
        daily_sales = self.df.groupby('order_date')['sales'].sum().sort_values(ascending=False).head(10)
        daily_sales.plot(kind='barh', ax=axes[1, 1], color='teal')
        axes[1, 1].set_title('Top 10 Days by Sales', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Sales ($)')
        axes[1, 1].set_ylabel('Date')

        plt.tight_layout()
        plt.savefig(f'{self.output_dir}/05_advanced_visualizations.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {self.output_dir}/05_advanced_visualizations.png")
        plt.close()

    def generate_insights(self):
        """
        Generate business insights and recommendations
        """
        print("\n" + "="*60)
        print("=== KEY BUSINESS INSIGHTS & RECOMMENDATIONS ===")
        print("="*60)

        # Top category
        top_category = self.df.groupby('category')['sales'].sum().idxmax()
        top_category_sales = self.df.groupby('category')['sales'].sum().max()

        # Top region
        top_region = self.df.groupby('region')['sales'].sum().idxmax()

        # Best performing segment
        top_segment = self.df.groupby('customer_segment')['profit'].sum().idxmax()

        # Optimal discount
        discount_profit = self.df.groupby('discount')['profit'].mean()
        optimal_discount = discount_profit.idxmax()

        insights = f"""

1. TOP PERFORMING CATEGORY
   → {top_category} generates the highest revenue: ${top_category_sales:,.2f}
   → Recommendation: Increase inventory and marketing for {top_category}

2. REGIONAL PERFORMANCE
   → {top_region} region shows strongest sales performance
   → Recommendation: Allocate more resources to {top_region} region

3. CUSTOMER SEGMENTATION
   → {top_segment} segment is most profitable
   → Recommendation: Develop targeted campaigns for {top_segment} customers

4. DISCOUNT STRATEGY
   → Optimal discount rate: {optimal_discount}%
   → Recommendation: Focus promotional discounts around {optimal_discount}%

5. SEASONALITY PATTERNS
   → Q4 shows significant sales spike (holiday season)
   → Recommendation: Prepare inventory and staffing for Q4 demand surge

6. GROWTH OPPORTUNITIES
   → Expand product lines in high-performing categories
   → Improve underperforming regions through targeted marketing
   → Optimize pricing strategy based on customer segments
        """

        print(insights)

        # Append to report
        with open(f'{self.output_dir}/sales_analysis_report.txt', 'a') as f:
            f.write("\n\nKEY BUSINESS INSIGHTS & RECOMMENDATIONS\n")
            f.write("="*60 + "\n")
            f.write(insights)

    def run_complete_analysis(self):
        """
        Execute complete analysis pipeline
        """
        print("\n" + "#"*60)
        print("#" + " "*58 + "#")
        print("#  E-COMMERCE SALES ANALYSIS & FORECASTING PROJECT  #")
        print("#" + " "*58 + "#")
        print("#"*60)

        # Step 1: Generate data
        self.generate_data(n_records=10000)

        # Step 2: Data overview
        self.data_overview()

        # Step 3: Descriptive statistics
        self.descriptive_statistics()

        # Step 4: EDA
        self.exploratory_data_analysis()

        # Step 5: Time series analysis
        monthly_data = self.time_series_analysis()

        # Step 6: Correlation analysis
        self.correlation_analysis()

        # Step 7: Hypothesis testing
        self.hypothesis_testing()

        # Step 8: Sales forecasting
        self.sales_forecasting(monthly_data)

        # Step 9: Advanced visualizations
        self.advanced_visualizations()

        # Step 10: Generate insights
        self.generate_insights()

        # Save dataset
        self.df.to_csv(f'{self.output_dir}/ecommerce_sales_data.csv', index=False)
        print(f"\n✓ Dataset saved: {self.output_dir}/ecommerce_sales_data.csv")

        print("\n" + "="*60)
        print("✓ ANALYSIS COMPLETE!")
        print(f"✓ All outputs saved in '{self.output_dir}/' directory")
        print("="*60 + "\n")

if __name__ == "__main__":
    analyzer = EcommerceSalesAnalysis()
    analyzer.run_complete_analysis()



############################################################
#                                                          #
#  E-COMMERCE SALES ANALYSIS & FORECASTING PROJECT  #
#                                                          #
############################################################

=== Generating E-commerce Sales Data ===
✓ Generated 10,000 records
✓ Date range: 2023-01-01 to 2024-12-30

=== DATA OVERVIEW ===

Dataset Shape: 10000 rows × 12 columns

Column Data Types:
order_id                    object
order_date          datetime64[ns]
category                    object
region                      object
customer_segment            object
quantity                     int64
unit_price                 float64
sales                      float64
discount                     int64
profit_margin              float64
profit                     float64
month                        int32
dtype: object

Missing Values:
order_id            0
order_date          0
category       